In [ ]:
import sys
import os

# Add src to sys.path to import modules
sys.path.append(os.path.abspath('../src'))

print("Added ../src to sys.path")

# Interactive Audit Testing

This notebook allows for real-time testing of the IFRS9 Automated Auditor without using web ports.

In [1]:
from config import CONFIG

CONFIG

{'llm_settings': {'provider': 'openai',
  'temperature': 0.0,
  'openai': {'model': 'gpt-4o-mini'},
  'google': {'model': 'models/gemini-pro-latest'}},
 'rag_settings': {'chunk_size': 1500,
  'chunk_overlap': 300,
  'document_language': 'Spanish'},
 'paths': {'input_csv': 'rcm_input.csv',
  'output_json': 'audit_results.json',
  'documents_folder': 'documents/'},
 'validation': {'enable_self_critique': True}}

In [2]:
# Cell 1: Load environment and initialize RcmAuditor
import os
from dotenv import load_dotenv
from rcm_engine import RcmAuditor
from config import CONFIG

load_dotenv()

# Initialize Auditor
auditor = RcmAuditor()
print("RcmAuditor initialized.")

# Ensure documents are indexed
auditor.initialize_rag()
print("RAG Index ready.")

RcmAuditor initialized.
Loading documents from documents/...
Loading Política de Previsionamiento_modificada sep 2025.pdf...
Loading Respuesta Memorando final Inspección 2025_Perdida Esperada NIIF.pdf...
Creating vector store with 58 chunks (Size: 1500, Overlap: 300)...
Processing batch 1/6 (10 chunks)...
Processing batch 2/6 (10 chunks)...
Processing batch 3/6 (10 chunks)...
Processing batch 4/6 (10 chunks)...
Processing batch 5/6 (10 chunks)...
Processing batch 6/6 (8 chunks)...
Index built successfully.
RAG Index ready.


In [3]:
# Cell 1.5: Dynamic Configuration (LLM Provider & Language)
from config import CONFIG
from llm_factory import reload_config_and_reinit
import os

print(f"Current Provider: {CONFIG['llm_settings']['provider']}")
print(f"Current Language: {auditor.rag_engine.doc_language}")

# Function to switch provider dynamically
def switch_provider(new_provider):
    CONFIG['llm_settings']['provider'] = new_provider
    # Re-initialize Auditor with new config
    global auditor
    auditor = RcmAuditor() # This uses the factory which reads the updated CONFIG
    auditor.initialize_rag()
    print(f"Switched to provider: {new_provider}")

# Example: Uncomment to switch to Google
# switch_provider('google')
# switch_provider('openai')

Current Provider: openai
Current Language: Spanish


In [4]:
# Cell 2: Define a specific test question manually
test_question = "Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?"#"Como se calcula la PD a 12 meses?"
print(f"Test Question: {test_question}")

Test Question: Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?


##  Run the retrieval step only and print retrieved PDF text chunks


In [5]:
print("Retrieving top 5 chunks...")

# Verify HyDE generation
print(f"Generating HyDE query for: '{test_question}'")
hyde_query = auditor.rag_engine.generate_search_query(test_question)
print(f"HyDE Output ({auditor.rag_engine.doc_language}):\n{hyde_query}\n")

retrieved_docs = auditor.rag_engine.retrieve(test_question, k=5)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:500] + "...") # Print first 500 chars

Retrieving top 5 chunks...
Generating HyDE query for: 'Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?'
HyDE Output (Spanish):
La estimación de la Probabilidad de Default (PD) a lo largo de la vida útil de las exposiciones está intrínsecamente relacionada con el enfoque de staging establecido por la NIIF 9. En este contexto, la PD vitalicia se calcula considerando la evolución del riesgo crediticio a lo largo del tiempo, lo que permite una evaluación más precisa de las pérdidas esperadas. Este enfoque es coherente con la duración esperada de las exposiciones, ya que se basa en la información histórica y en las expectativas futuras del comportamiento crediticio de los deudores. Además, la segmentación de las exposiciones en diferentes etapas (Stage 1, Stage 2 y Stage 3) permite ajustar la estimación de la PD en función del deterioro del crédito, asegurando que las provisiones reflejen adecuadamente el riesgo inherente a c

## Run the full process_row logic and print AI Answer AND Critique Score


In [6]:

# Mock a row data structure
mock_row = {
    'Control Reference': 'Test-Ref-001',
    'Test Procedure': test_question
}

print("Running full process_row...")
result = auditor.process_row(mock_row)

print("\n=== AI Answer ===")
print(result['AI_Answer'])

print("\n=== Compliance Verdict ===")
print(f"Verdict: {result.get('Compliance_Verdict', 'N/A')}")
print("\n=== Validation ===")
print(f"Score: {result['Validation_Score']}")
print(f"Reasoning: {result['Validation_Reasoning']}")

Running full process_row...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: Test-Ref-001. Question:  (Procedure: ...
HyDE Search Query: La estimación de la Probabilidad de Default (PD) a...

=== AI Answer ===
The lifetime Probability of Default (PD) estimation is explained in the context as follows:

1. The methodology for determining the PD for Stage 2 is based on a lifetime horizon, which is consistent with the expected life of exposures. Specifically, it states: "Stage 2 = EAD x (PD (Lifetime) x FL) x LGD" [Page 8].

2. The context also mentions that a curve of default is constructed to capture the accumulated probabilities of default from the date of analysis until the extinction of the portfolio, which aligns with the concept of lifetime PD. This is further supported by the statement: "se construye una curva de default que recoge las probabilidades de incumplimiento acumuladas a partir de una cartera, desde la fecha de análisis hasta la extinción de la mism

In [ ]:
# Test Client Summary Generation
print("Testing Client Summary Generation...")
auditor.generate_client_summary()

import os
if os.path.exists("outputs/client_summary.txt"):
    with open("outputs/client_summary.txt", "r", encoding="utf-8") as f:
        print("\n=== Client Summary ===")
        print(f.read())
else:
    print("Client summary file not found.")

## Run Full Audit (Batch Process)
This section mimics the `run_audit.py` script, processing the entire CSV file and saving results to JSON.

In [7]:
# Cell 4: Run Full Audit on CSV
import pandas as pd
import json
import time
import os

print("Starting Full Audit Process in Notebook...")

# Load Input
input_csv = CONFIG['paths']['input_csv']
if not os.path.exists(input_csv):
    print(f"Error: Input file {input_csv} not found.")
else:
    print(f"Reading input from {input_csv}...")
    try:
        df = pd.read_csv(input_csv, sep=';', encoding='latin-1')
        
        results = []
        total_rows = len(df)
        print(f"Processing {total_rows} rows...")

        for idx, row in df.iterrows():
            print(f"Processing row {idx + 1}/{total_rows}...")
            try:
                row_dict = row.to_dict()
                res = auditor.process_row(row_dict)
                results.append(res)
            except Exception as e:
                print(f"Error processing row {idx + 1}: {e}")
                # Add error info to result
                err_row = row.to_dict()
                err_row['AI_Answer'] = f"Error: {e}"
                results.append(err_row)
            
            # Polite delay between rows to avoid hitting rate limits
            time.sleep(1)

        # Save Results
        output_json = CONFIG['paths']['output_json']
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"Audit complete. Results saved to {output_json}")
        
    except Exception as e:
        print(f"Error reading CSV or saving results: {e}")

Starting Full Audit Process in Notebook...
Reading input from rcm_input.csv...
Processing 68 rows...
Processing row 1/68...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: 1.1. Question: Is there a formal poli...
HyDE Search Query: Sí, existe una política formal para el cálculo de ...
Processing row 2/68...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: 1.2. Question: Are the IFRS 9 policy ...
HyDE Search Query: La política y los procedimientos relacionados con ...
Processing row 3/68...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: 1.3. Question: Are there formal contr...
HyDE Search Query: Sí, existen controles y procedimientos formales es...
Processing row 4/68...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: 1.4. Question: Is documentation writt...
HyDE Search Query: La documentación revisada cumple con los criterios...
Processing row 5/68...
DEBUG: Generating HyDE query in Spanish.

## Step 5: Expert Comparison (Validation)
Compare the AI-generated answers against the expert ground truth.
This step runs the logic from `validate_audit.py`.

In [ ]:
# Run Validation / Expert Comparison
from validate_audit import validate_audit

print("Running Expert Comparison...")
validate_audit()
print("Done. Check outputs/validation_comparison_report.csv")